In [29]:
#imports
from pydub import AudioSegment
from pydub.utils import make_chunks
import os
from glob import glob
import time

#audio feature extraction library
import librosa
from librosa import feature

import numpy as np
import matplotlib.pyplot as plt

#model
import tensorflow as tf
from spela.spectrogram import Spectrogram 
from spela.melspectrogram import Melspectrogram

from sklearn.model_selection import train_test_split
import soundfile as sf

In [30]:
def findAllAudioFilePaths():
    audioFilesPaths = [y for x in os.walk("Dataset/Youtube Speech Dataset/Dataset") for y in glob(os.path.join(x[0], '*.wav'))]
    return audioFilesPaths

In [31]:
speakers = ["Obama", "Hillary", "Ivanka", "Trump", "No Speaker", "Modi", "Xi-Jinping", "Chadwick-Boseman"]

def speakerToLabel(speakerName):
    index = speakers.index(speakerName)
    
    if(index == -1):
        print("Speaker not found: {}".format(speakerName))
    
    return index

In [32]:
def getSpeakerAndAudio(audioPaths):
    audio_Paths = []
    labels = []
    uniqueSpeakers = {}

    for audioPath in audioPaths:
        speakerName = audioPath.split("/")[3]

        audioLength = librosa.get_duration(filename=audioPath)
        
        if audioLength == 1.0:
            audio_Paths.append(audioPath)
            labels.append(speakerToLabel(speakerName))
            uniqueSpeakers[speakerName] = uniqueSpeakers.get(speakerName, 0) + 1
        else:
            print("Audio clip discarded, actual length = {}".format(audioLength))
    
    return audio_Paths, labels, uniqueSpeakers

In [33]:
 #https://towardsdatascience.com/how-i-understood-what-features-to-consider-while-training-audio-files-eedfb6e9002b
        

def getFeatures(audio_Paths):
    
    data_X = []
    
    for path in audio_Paths:
        
        audioFeatureArray = []        
        y, sr = librosa.load(path)
        
        #mfcc
        mfccArray = librosa.feature.mfcc(y=y, sr=sr)
        audioFeatureArray.append(mfccArray.flatten())
        audioFeatureNumpyArray = np.array(audioFeatureArray)
        
        
        #zero_crossing_rate
        zeroCrossingArray = librosa.feature.zero_crossing_rate(y=y)
        np.append(audioFeatureNumpyArray, zeroCrossingArray.flatten())
        
        #spectral_rolloff
        spectralRollOffArray = librosa.feature.spectral_rolloff(y=y, sr=sr)
        np.append(audioFeatureNumpyArray, spectralRollOffArray.flatten())
        
        data_X.append(audioFeatureNumpyArray.flatten())
        
        
    return data_X

In [34]:
audioPaths = findAllAudioFilePaths()
audio_Paths, labels, uniqueSpeakers = getSpeakerAndAudio(audioPaths)
 
    
#print("Speakers: {}".format(uniqueSpeakers))

print("")
print("-------------------------")
for speaker in uniqueSpeakers: 
    print("Speaker: {}, length: {} minutes".format(speaker, int(uniqueSpeakers[speaker])/60 )) 
print("-------------------------")
print("")


print("Total Dataset size: {}".format(len(audio_Paths)))

startTime = time.time()
data_X = getFeatures(audio_Paths)
endTime = time.time()

print("Time to extract Features {} minutes".format( (endTime-startTime)/60))

print("X data: {}".format(len(data_X)))
print("Y data: {}".format(len(labels)))




Audio clip discarded, actual length = 0.221
Audio clip discarded, actual length = 0.4819954648526077
Audio clip discarded, actual length = 0.66
Audio clip discarded, actual length = 0.3
Audio clip discarded, actual length = 0.6059863945578231
Audio clip discarded, actual length = 0.12598639455782312
Audio clip discarded, actual length = 0.7489795918367347

-------------------------
Speaker: No Speaker, length: 9.65 minutes
Speaker: Hillary, length: 57.53333333333333 minutes
Speaker: Ivanka, length: 17.916666666666668 minutes
Speaker: Xi-Jinping, length: 11.183333333333334 minutes
Speaker: Modi, length: 32.4 minutes
Speaker: Chadwick-Boseman, length: 27.083333333333332 minutes
Speaker: Obama, length: 19.466666666666665 minutes
Speaker: Trump, length: 41.6 minutes
-------------------------

Total Dataset size: 13010
Time to extract Features 11.66484010219574 minutes
X data: 13010
Y data: 13010


In [35]:
train_X, test_X, train_Y, test_Y = train_test_split(data_X, labels, test_size=0.2)

train_x = np.array(train_X)
train_y = np.array(train_Y)
test_x = np.array(test_X)
test_y = np.array(test_Y)

train_y = tf.keras.utils.to_categorical(train_y)
test_y = tf.keras.utils.to_categorical(test_y)


In [54]:
# create a model
#from tf.keras.layers import Dense

def create_model():
    model = tf.keras.Sequential()
    
    
    model.add(tf.keras.layers.Dense(12,input_shape= train_x.shape, activation='relu'))
    model.add(tf.keras.layers.Dense(8, activation='relu'))
    model.add(tf.keras.layers.Dense(len(speakers), activation="softmax"))
                          
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=3e-4)
            , loss = "categorical_crossentropy"
            , metrics = ["accuracy"])
                          
    
    
    return model

In [55]:
model = create_model()

model.fit(x=train_x, y=train_y, epochs=50, validation_data=(test_x, test_y))

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 10408, 12)         10572     
_________________________________________________________________
dense_19 (Dense)             (None, 10408, 8)          104       
_________________________________________________________________
dense_20 (Dense)             (None, 10408, 8)          72        
Total params: 10,748
Trainable params: 10,748
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
326/326 [==============================] - 1s 2ms/step - loss: 3.1337 - accuracy: 0.2272 - val_loss: 2.0595 - val_accuracy: 0.2759
Epoch 2/50
326/326 [==============================] - 0s 1ms/step - loss: 2.0055 - accuracy: 0.2635 - val_loss: 1.8084 - val_accuracy: 0.2763
Epoch 3/50
326/326 [==============================] - 0s 1ms/step - loss: 1.7305 - accuracy: 0.3

Epoch 48/50
326/326 [==============================] - 0s 1ms/step - loss: 0.6794 - accuracy: 0.7178 - val_loss: 0.7850 - val_accuracy: 0.7025
Epoch 49/50
326/326 [==============================] - 0s 1ms/step - loss: 0.6875 - accuracy: 0.7156 - val_loss: 0.8148 - val_accuracy: 0.6998
Epoch 50/50
326/326 [==============================] - 0s 1ms/step - loss: 0.6715 - accuracy: 0.7239 - val_loss: 0.7865 - val_accuracy: 0.7014


In [56]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 10408, 12)         10572     
_________________________________________________________________
dense_19 (Dense)             (None, 10408, 8)          104       
_________________________________________________________________
dense_20 (Dense)             (None, 10408, 8)          72        
Total params: 10,748
Trainable params: 10,748
Non-trainable params: 0
_________________________________________________________________
